In [2]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
from nltk.corpus import stopwords


data = pd.read_csv(r"../../data/sentiment_analysis_classifier/tweet_sentiments.csv")





,Text,Sentiment,SimpleSent,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park! ...,Positive,Positive,2023-01-15 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning. ...,Negative,Negative,2023-01-15 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout! ðŸ’ª ...,Positive,Positive,2023-01-15 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway! ...,Positive,Positive,2023-01-15 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight. ...,Neutral,Neutral,2023-01-15 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,Positive,2017-08-18 18:20,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20,39,UK,2017,8,18,18
728,Attending a surprise birthday party organized ...,Happy,Positive,2018-06-22 14:15,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25,48,USA,2018,6,22,14
729,Successfully fundraising for a school charity ...,Happy,Positive,2019-04-05 17:30,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22,42,Canada,2019,4,5,17
730,"Participating in a multicultural festival, cel...",Happy,Positive,2020-02-29 20:45,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21,43,UK,2020,2,29,20


In [2]:
'''
    Natural language preprocessing

    Remove punctuation, make all words lowercase, and lemmatize
'''

from nltk.corpus import wordnet
from textblob import Word
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string

'''
    NLTK has a model to tag words as adjectives, nouns, etc,
    but NLTK uses wordnet for lemmatization. wordnet only uses
    four possible tags, while NLTK returns tons of unique ones

    This function transforms NLTK tags to wordnet tags for lemmatization
'''
def nltk_tag_to_wordnet(tag: str) -> str:
    if tag[0] == "J":
        return wordnet.ADJ
    elif tag[0] == "V":
        return wordnet.VERB
    elif tag[0] == "N":
        return wordnet.NOUN
    elif tag[0] == "R":
        return wordnet.ADV
    else:
        return ""
    
'''
    Remove non-alphabetical characters and punctuation
'''
def keep_only_alphabetic(s: str) -> str:
    temp = "".join([" " if i in string.punctuation else i for i in s])
    return "".join([i for i in temp if (ord(i) <= 90 and ord(i) >= 65) or (ord(i) <= 122 and ord(i) >= 97) or i.isspace()])

'''
    Take a string of text, tokenize it, and return a list of lemmatized tokens
'''
def lemmatize_words(s: str) -> list[str]:
    lemmer = nltk.stem.WordNetLemmatizer()
    words = [i.lower() for i in word_tokenize(s)]       # tokenize and lowercase
    words = [i for i in words if i not in stopwords.words("english")]   # remove stopwords
    words = list(filter(lambda x: nltk_tag_to_wordnet(x[1]) !="", pos_tag(words)))  # remove invalid lemmatization words and tags
    words = [lemmer.lemmatize(i[0], nltk_tag_to_wordnet(i[1])) for i in words]  #  lemmatize words
    return words

'''
    Combine all functions above to pre-process strng
'''
def pre_process_text(text: str) -> str:
    s = keep_only_alphabetic(text)
    lemmatized = lemmatize_words(s)
    return " ".join(lemmatized)






In [3]:
data["Text"] = data.apply(lambda x: keep_only_alphabetic(x["Text"]), axis=1)
data

,Text,Sentiment,SimpleSent,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park ...,Positive,Positive,2023-01-15 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning ...,Negative,Negative,2023-01-15 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout ...,Positive,Positive,2023-01-15 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway ...,Positive,Positive,2023-01-15 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight ...,Neutral,Neutral,2023-01-15 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,Positive,2017-08-18 18:20,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20,39,UK,2017,8,18,18
728,Attending a surprise birthday party organized ...,Happy,Positive,2018-06-22 14:15,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25,48,USA,2018,6,22,14
729,Successfully fundraising for a school charity ...,Happy,Positive,2019-04-05 17:30,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22,42,Canada,2019,4,5,17
730,Participating in a multicultural festival cel...,Happy,Positive,2020-02-29 20:45,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21,43,UK,2020,2,29,20


In [4]:
def to_class(s):
    match s:
        case "Positive":
            return 2
        case "Negative":
            return 0
        case "Neutral":
            return 1
    

In [5]:
data["SimpleSent"] = data.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
data

,Text,Sentiment,SimpleSent,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park ...,Positive,2,2023-01-15 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning ...,Negative,0,2023-01-15 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout ...,Positive,2,2023-01-15 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway ...,Positive,2,2023-01-15 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight ...,Neutral,1,2023-01-15 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,2,2017-08-18 18:20,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20,39,UK,2017,8,18,18
728,Attending a surprise birthday party organized ...,Happy,2,2018-06-22 14:15,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25,48,USA,2018,6,22,14
729,Successfully fundraising for a school charity ...,Happy,2,2019-04-05 17:30,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22,42,Canada,2019,4,5,17
730,Participating in a multicultural festival cel...,Happy,2,2020-02-29 20:45,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21,43,UK,2020,2,29,20


In [6]:
toDrop = ["Timestamp", "User", "Platform", "Retweets", "Likes", "Country", "Year", "Month", "Day", "Hour"]

data = data.drop(toDrop, axis=1)
data

,Text,Sentiment,SimpleSent,Hashtags
0,Enjoying a beautiful day at the park ...,Positive,2,#Nature #Park
1,Traffic was terrible this morning ...,Negative,0,#Traffic #Morning
2,Just finished an amazing workout ...,Positive,2,#Fitness #Workout
3,Excited about the upcoming weekend getaway ...,Positive,2,#Travel #Adventure
4,Trying out a new recipe for dinner tonight ...,Neutral,1,#Cooking #Food
...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,2,#ScienceFairWinner #HighSchoolScience
728,Attending a surprise birthday party organized ...,Happy,2,#SurpriseCelebration #HighSchoolFriendship
729,Successfully fundraising for a school charity ...,Happy,2,#CommunityGiving #HighSchoolPhilanthropy
730,Participating in a multicultural festival cel...,Happy,2,#CulturalCelebration #HighSchoolUnity


In [7]:
data["text_tb_pol"] = data.apply(lambda x: TextBlob(x["Text"]).polarity, axis=1)
data["text_tb_sub"] = data.apply(lambda x: TextBlob(x["Text"]).subjectivity, axis=1)




In [8]:
analyzer = SentimentIntensityAnalyzer()

data["text_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["Text"]), axis=1)
data["text_vader_neg"] = data.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
data["text_vader_neu"] = data.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
data["text_vader_pos"] = data.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
data = data.drop(["text_vader_scores"], axis=1)




data


,Text,Sentiment,SimpleSent,Hashtags,text_tb_pol,text_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos
0,Enjoying a beautiful day at the park ...,Positive,2,#Nature #Park,0.675000,0.800000,0.000,0.407,0.593
1,Traffic was terrible this morning ...,Negative,0,#Traffic #Morning,-1.000000,1.000000,0.437,0.563,0.000
2,Just finished an amazing workout ...,Positive,2,#Fitness #Workout,0.600000,0.900000,0.000,0.513,0.487
3,Excited about the upcoming weekend getaway ...,Positive,2,#Travel #Adventure,0.375000,0.750000,0.000,0.676,0.324
4,Trying out a new recipe for dinner tonight ...,Neutral,1,#Cooking #Food,0.136364,0.454545,0.000,1.000,0.000
...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,2,#ScienceFairWinner #HighSchoolScience,0.700000,0.900000,0.000,0.607,0.393
728,Attending a surprise birthday party organized ...,Happy,2,#SurpriseCelebration #HighSchoolFriendship,0.600000,0.600000,0.000,0.331,0.669
729,Successfully fundraising for a school charity ...,Happy,2,#CommunityGiving #HighSchoolPhilanthropy,0.516667,0.383333,0.000,0.474,0.526
730,Participating in a multicultural festival cel...,Happy,2,#CulturalCelebration #HighSchoolUnity,1.000000,1.000000,0.000,0.485,0.515


In [9]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd

def tf_idf_vectorize(df: pd.DataFrame, corpus: pd.Series, vocabulary: list[str]) -> tuple[list[str], pd.DataFrame]:
    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3), vocabulary=vocabulary)
    features = vectorizer.fit_transform(corpus).toarray()
    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(features, columns=headers)
    #final = pd.concat([df, feature_frame], axis=1)
    return (names, feature_frame)


def get_full_vocabulary(corpus: pd.Series) -> list[str]:
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stopwords.words("english"), max_features=500, ngram_range=(1,3))
    vectorizer.fit_transform(corpus).toarray()
    return vectorizer.get_feature_names_out()


def add_tf_idf_vector(x: pd.DataFrame, partial_corpus: pd.Series, full_corpus: pd.Series) -> pd.DataFrame:
    temp = pd.concat((x, partial_corpus), axis=1).reset_index()
    # first, we need a vocabulary from the entire dataset
    vocabulary = get_full_vocabulary(full_corpus)

    tf_idf_vector = tf_idf_vectorize(temp, temp[partial_corpus.name], vocabulary=vocabulary)[1].reset_index()
    temp = temp.reset_index()
    temp = pd.concat((temp, tf_idf_vector), axis=1).drop([partial_corpus.name, "index"], axis=1)
    return temp



In [11]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold

classes = data["SimpleSent"]
features = data[["text_tb_pol",  "text_vader_pos", "text_vader_neg", "text_vader_neu"]]



test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], data["Text"].iloc[train_index], data["Text"])
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], data["Text"].iloc[test_index], data["Text"])
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    ensemble.fit(x_train, y_train)
    y_pred = ensemble.predict(x_test)
    test_total += accuracy_score(y_test, y_pred)
    run_count += 1

print(f"Test accuracy on {folds}-fold cross-validation: {test_total / run_count}")





0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Test accuracy on 5-fold cross-validation: 0.8332401453732178


In [12]:
'''
    POLARITY ONLY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold

classes = data["SimpleSent"]
features = data[["text_tb_pol",  "text_vader_pos", "text_vader_neg", "text_vader_neu"]]



test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()

    x_train = x.iloc[train_index]
    x_train = scaler.fit_transform(x_train)


    x_test = x.iloc[test_index]
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    ensemble.fit(x_train, y_train)
    y_pred = ensemble.predict(x_test)
    test_total += accuracy_score(y_test, y_pred)
    run_count += 1

print(f"Test accuracy on {folds}-fold cross-validation: {test_total / run_count}")





0
ensemble fitting
1
ensemble fitting
2
ensemble fitting
3
ensemble fitting
4
ensemble fitting
Test accuracy on 5-fold cross-validation: 0.8277700121144347


In [13]:
'''
    TF-IDF ONLY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold

classes = data["SimpleSent"]
features = data[["text_tb_pol"]]



test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], data["Text"].iloc[train_index], data["Text"]).drop("text_tb_pol", axis=1)
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], data["Text"].iloc[test_index], data["Text"]).drop("text_tb_pol", axis=1)
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    ensemble.fit(x_train, y_train)
    y_pred = ensemble.predict(x_test)
    test_total += accuracy_score(y_test, y_pred)
    run_count += 1

print(f"Test accuracy on {folds}-fold cross-validation: {test_total / run_count}")




0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Test accuracy on 5-fold cross-validation: 0.7486068399962725


In [ ]:
import pickle


with open("TWEETS_ENSEMBLE.pkl", "wb") as file:
    pickle.dump(ensemble, file)

